In [0]:
pip install kaggle

In [0]:
mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akshaygahlot","key":"5fdaf7c8ff7c82007299c33dc6dd1ad4"}'}

In [0]:
cp kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c digit-recognizer

 55% 5.00M/9.16M [00:00<00:00, 32.5MB/s]
100% 9.16M/9.16M [00:00<00:00, 45.0MB/s]
 99% 6.00M/6.09M [00:00<00:00, 61.9MB/s]
100% 6.09M/6.09M [00:00<00:00, 56.2MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 216MB/s]


In [0]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.metrics import FalseNegatives
from keras.metrics import FalsePositives
import os
import zipfile
import gzip
import PIL
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from numpy import asarray
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
import tensorflow as tf
from tensorflow.python.framework import ops
tf.compat.v1.disable_eager_execution()
from sklearn import linear_model
import pandas as pd

Using TensorFlow backend.


In [0]:
zip = zipfile.ZipFile("/content/test.csv.zip","r")
unzipped=zip.extractall()

In [0]:
zip = zipfile.ZipFile("/content/train.csv.zip","r")
unzipped=zip.extractall()

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.python.framework import ops
tf.compat.v1.disable_eager_execution()
from sklearn import linear_model
import pandas as pd
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
train_orig_X = (train.iloc[:,1:].values).astype('float32') 
train_orig_Y = train.iloc[:,0].values.astype('int32') 
test_orig_X = test.values.astype('float32')
train_orig_X.resize(42000,28,28,1)
train_orig_X/=255
test_orig_X.resize(28000,28,28,1)
test_orig_X/=255
# file='train-images.idx3-ubyte'
# train_orig_X=idx2numpy.convert_from_file(file)              #training images
#train_orig_X.resize(42000,784)
#train_orig_X=train_orig_X/255                               #standardization 
# file='train-labels.idx1-ubyte'
# train_orig_Y=idx2numpy.convert_from_file(file)              #train images labels
# print(train_orig_Y.shape)
# log_reg=linear_model.LogisticRegression(max_iter=1000)
# log_reg.fit(train_orig_X,train_orig_Y)
# print(log_reg.score(train_orig_X,train_orig_Y))             #score:93.93
# file='t10k-images.idx3-ubyte'
# test_orig_X=idx2numpy.convert_from_file(file)
#test_orig_X.resize(28000,784)
#test_orig_X=test_orig_X/255
# file='t10k-labels.idx1-ubyte'
# test_orig_Y=idx2numpy.convert_from_file(file)
# print(log_reg.score(test_orig_X,test_orig_Y))               #score:92.56
# train_orig_Y_one_hot=np.eye(10)[train_orig_Y]
# train_orig_Y_one_hot.resize(10,60000)
# print(train_orig_Y)
# print(train_orig_Y_one_hot)

def encoding(Y):
    y_enc=tf.one_hot(Y,10,1)
    with tf.compat.v1.Session() as sess:
        one_hot=sess.run(y_enc)
        sess.close()
        return one_hot

# ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
# X=tf.compat.v1.placeholder(tf.float32,[784,None],name="X")
# Y=tf.compat.v1.placeholder(tf.float32,[10,None],name="Y")
# W1=tf.compat.v1.get_variable("W1",[32,784],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))
# b1=tf.compat.v1.get_variable("b1",[32,1],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))
# W2=tf.compat.v1.get_variable("W2",[10,32],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))
# b2=tf.compat.v1.get_variable("b2",[10,1],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))
# # W3=tf.compat.v1.get_variable("W3",[10,15],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))
# # b3=tf.compat.v1.get_variable("b3",[10,1],initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform", seed=0))

# Z1=tf.add(tf.matmul(W1,X),b1)
# A1=tf.nn.relu(Z1)
# Z2=tf.add(tf.matmul(W2,A1),b2)
# # A2=tf.nn.relu(Z2)
# # Z3=tf.add(tf.matmul(W3,A2),b3)

# cost = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(a=Z2), labels=tf.stop_gradient(tf.transpose(a=Y))))

# optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

# init=tf.compat.v1.global_variables_initializer()
# with tf.compat.v1.Session() as sess:
#     sess.run(init)
#     for i in range(2000):
#         sess.run([optimizer,cost],feed_dict={X:np.transpose(train_orig_X),Y:(encoding(train_orig_Y).T)})
#         #print(W1.eval())
#     correct_prediction = tf.equal(tf.argmax(input=Z2), tf.argmax(input=Y))
#     accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, "float"))
#     print("Train Accuracy:", accuracy.eval({X: np.transpose(train_orig_X), Y: encoding(train_orig_Y).T}))
#     print("Test Accuracy:",accuracy.eval({X: np.transpose(test_orig_X), Y: encoding(test_orig_Y).T}))
# Around 20 mins of train time:
# Train Accuracy: 0.98725
# Test Accuracy: 0.9692
models=[0]*15
for j in range(len(models)):
  models[j]=Sequential()
  # model.add(Dense(units=512,activation='relu',input_shape=(784,)))
  # model.add(Dense(units=100,activation='relu'))
  # model.add(Dense(units=10,activation='softmax'))
  # model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
  # model.fit((train_orig_X), encoding(train_orig_Y), batch_size=128, epochs=40, verbose=1, validation_split=0)
  # #model.evaluate(test_orig_X,encoding(test_orig_Y))
  models[j].add(Conv2D(32,(3,3),strides=(1,1),name="first",activation='relu',input_shape=(28,28,1)))
  models[j].add(BatchNormalization())
  models[j].add(AveragePooling2D((2,2),strides=(2,2)))
  models[j].add(Conv2D(64,(5,5),strides=(1,1),activation='relu'))
  models[j].add(BatchNormalization())
  models[j].add(AveragePooling2D((2,2),strides=(2,2)))
  # model.add(Conv2D(256,(3,3),strides=(1,1),activation='relu'))
  # model.add(BatchNormalization())
  # model.add(AveragePooling2D((2,2),strides=(2,2)))
  models[j].add(Flatten())
  models[j].add(Dense(units=512,activation='relu'))
  models[j].add(Dense(units=256,activation='relu'))
  models[j].add(Dense(units=10,activation='softmax'))
  models[j].compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

  aug_data=ImageDataGenerator(rotation_range=10,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1)
  aug_data.fit(train_orig_X)
  models[j].fit(aug_data.flow(train_orig_X,encoding(train_orig_Y), batch_size=128), epochs=100, verbose=1)
# predictions = model.predict_classes(test_orig_X, verbose=0)
# submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),"Label": predictions})
# submissions.to_csv("DR.csv", index=False, header=True)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/100
329/329 [==============================] - 17s 53ms/step - loss: 0.2034 - accuracy: 0.9350
Epoch 2/100
329/329 [==============================] - 12s 35ms/step - loss: 0.0799 - accuracy: 0.9754
Epoch 3/100
329/329 [==============================] - 11s 34ms/step - loss: 0.0580 - accuracy: 0.9817
Epoch 4/100
329/329 [==============================] - 11s 35ms/step - loss: 0.0513 - accuracy: 0.9840
Epoch 5/100
329/329 [==============================] - 12s 35ms/step - loss: 0.0466 - accuracy: 0.9854
Epoch 6/100
329/329 [==============================] - 12s 35ms/step - loss: 0.0396 - accuracy: 0.9877
Epoch 7/100
329/329 [==============================] - 12s 35ms/step - loss: 0.0375 - accuracy: 0.9881
Epoch 8/100
329/329 [==============================] - 11s 35ms/step - loss: 0.0347 - accuracy: 0.9891
Epoch 9/100
329/329 [==============================] - 12s 35ms/step - loss: 0.0329 - accuracy:

In [0]:
results = np.zeros( (test_orig_X.shape[0],10) ) 
for j in range(len(models)):
    # try:
    results = results + models[j].predict(test_orig_X)
    # except:
    # continue
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("MNIST-CNN-ENSEMBLE.csv",index=False)

In [0]:
# for j in range(len(models)):
#   models[j].fit(aug_data.flow(train_orig_X,encoding(train_orig_Y), batch_size=128), epochs=20, verbose=1)

Epoch 1/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0080 - accuracy: 0.9975
Epoch 2/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 3/20
329/329 [==============================] - 11s 34ms/step - loss: 0.0085 - accuracy: 0.9975
Epoch 4/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0090 - accuracy: 0.9972
Epoch 5/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0062 - accuracy: 0.9980
Epoch 6/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0101 - accuracy: 0.9974
Epoch 7/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0071 - accuracy: 0.9981
Epoch 8/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0074 - accuracy: 0.9978
Epoch 9/20
329/329 [==============================] - 11s 33ms/step - loss: 0.0088 - accuracy: 0.9975
Epoch 10/20
329/329 [==============================] - 11s 33ms/step - loss: 0.006

In [0]:
results = np.zeros( (test_orig_X.shape[0],10) ) 
for j in range(len(models)):
    # try:
    results = results + models[j].predict(test_orig_X)
    # except:
    # continue
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("MNIST-CNN-ENSEMBLE.csv",index=False)

In [0]:
print(type(models[7]))
print(type(models[8]))

<class 'keras.engine.sequential.Sequential'>
<class 'keras.engine.sequential.Sequential'>
